In [1]:
!pip install pandas
!pip install html5lib
!pip install lxml
!pip install BeautifulSoup4

     |████████████████████████████████| 5.8MB 28.3MB/s eta 0:00:01
     |████████████████████████████████| 112kB 7.6MB/s eta 0:00:01


In [2]:
from pandas.io.html import read_html

Getting all tables from the wikipedia page (in this case there's only one)

In [3]:
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

wikitables = read_html(page, attrs=("class","wikitable"))
print ("Extracted {num} wikitables".format(num=len(wikitables)))

Extracted 1 wikitables


In [4]:
wikitables[0].head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
wikitables[0].shape

(287, 3)

In [6]:
df_Toronto = wikitables[0]
df_Toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Dropping any rows with "Not assigned" in the Borough column

In [7]:
# Get names of indexes for which column Neighbourhood has value "Not assigned"
indexN = df_Toronto[ df_Toronto['Borough'] == "Not assigned" ].index
 
# Delete these row indexes from dataFrame
df_Toronto.drop(indexN , inplace=True)

In [8]:
df_Toronto.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


Combining neighbourhoods that all have the same postcode into one row & re-indexing

In [9]:
df_T = df_Toronto.groupby(["Postcode", "Borough"])["Neighbourhood"].apply(", ".join).to_frame(name = 'Neighbourhood').reset_index()

In [10]:
df_T.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
df_T.shape

(103, 3)

If any neighbourhoods are "Not assigned" I replace them with NaN and then use ffill to fill them with the value from the previous column, in this case the Borough.

In [14]:
import numpy as np
df_T.replace(to_replace ="Not assigned", value = np.nan) 

df_T.ffill(axis=1)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [15]:
df_T.tail(20)

,Postcode,Borough,Neighbourhood
83,M6R,West Toronto,"Parkdale, Roncesvalles"
84,M6S,West Toronto,"Runnymede, Swansea"
85,M7A,Downtown Toronto,Queen's Park
86,M7R,Mississauga,Canada Post Gateway Processing Centre
87,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
88,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto"
89,M8W,Etobicoke,"Alderwood, Long Branch"
90,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
91,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."
92,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen..."


In [16]:
df_T.shape

(103, 3)